
# 🔌 Redis Bus Demo

_Date generated: 2025-09-04_

This notebook demonstrates a **pub/sub and stream workflow with Redis** for a trading bus.

**What you get**
- Connect to Redis (local or container)
- Simple **pub/sub** messaging demo
- **Redis Streams** producer/consumer example
- Mini trade/order flow simulation


## 0) Setup

In [ ]:

import os, time, json
import redis

REDIS_HOST = os.getenv("REDIS_HOST","localhost")
REDIS_PORT = int(os.getenv("REDIS_PORT",6379))
r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, decode_responses=True)

print("Connected:", r.ping())


## 1) Pub/Sub Messaging

In [ ]:

# Publisher
channel = "orders"

def publisher(n=5):
    for i in range(n):
        msg = {"order_id": i, "symbol":"AAPL", "qty":100+i, "side":"BUY"}
        r.publish(channel, json.dumps(msg))
        print("Published", msg)
        time.sleep(0.2)

# Subscriber (blocking read for demo)
def subscriber():
    pubsub = r.pubsub()
    pubsub.subscribe(channel)
    print("Subscribed to", channel)
    for m in pubsub.listen():
        if m["type"]=="message":
            data = json.loads(m["data"])
            print("Received:", data)
            break

# Run publisher in background in real use, here just sequentially


## 2) Redis Streams Example

In [ ]:

stream = "trade_stream"

def producer(n=5):
    for i in range(n):
        ev = {"trade_id": i, "symbol":"MSFT", "px":str(100+0.1*i), "qty":str(50)}
        r.xadd(stream, ev)
        print("Produced", ev)

def consumer(block_ms=1000):
    last_id = "0-0"
    while True:
        resp = r.xread({stream:last_id}, block=block_ms, count=1)
        if resp:
            _, events = resp[0]
            for eid, fields in events:
                print("Consumed", eid, fields)
                last_id = eid
            break

# Run producer then consumer to demo


## 3) Mini Trade Bus Simulation

In [ ]:

# Simple demo: orders published -> risk check -> execution -> fill

def trade_bus_demo():
    # Order in stream
    order = {"order_id":"T1","symbol":"TSLA","side":"SELL","qty":"200"}
    r.xadd("orders_stream", order)
    print("Order submitted:", order)

    # Risk check reads, adds risk flag, pushes to next stream
    resp = r.xread({"orders_stream":"0-0"}, count=1)
    _, events = resp[0]
    for eid, fields in events:
        fields["risk_checked"] = "1"
        r.xadd("risk_stream", fields)
        print("Risk checked ->", fields)

    # Execution reads, fills, pushes fill
    resp = r.xread({"risk_stream":"0-0"}, count=1)
    _, events = resp[0]
    for eid, fields in events:
        fill = dict(fields); fill["status"]="FILLED"; fill["px"]="250.0"
        r.xadd("fills_stream", fill)
        print("Executed ->", fill)

    # Backoffice reads fills
    resp = r.xread({"fills_stream":"0-0"}, count=1)
    print("Backoffice received:", resp)

trade_bus_demo()
